In [1]:
!pip install datasets
!pip install transformers -U
!pip install accelerate -U
!pip install trl
!pip install bitsandbytes #for quantization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.9 MB/s eta 0:00:00


In [2]:
!pip install peft #parameter efficinet fine training (allow us to use LoRA)

In [3]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [4]:
import torch #hugging face is a wrapper on top of pytorch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
device

device(type='cuda')

In [6]:
from datasets import load_dataset
DATASET_NAME = "ChrisHayduk/Llama-2-SQL-Dataset"
dataset = load_dataset(DATASET_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/631 [00:00<?, ?B/s]

data/train-00000-of-00001-922416e34c5bc7(…):   0%|          | 0.00/9.11M [00:00<?, ?B/s]

data/eval-00000-of-00001-6907aec719559d7(…):   0%|          | 0.00/1.01M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/70719 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/7858 [00:00<?, ? examples/s]

# There is a trainingg set to trtain the model and evlaution set to test the model for the dataset on hugginface

quality of fine tuning data is a lot more important than quantity
1000 is plenty

In [7]:
full_training_dataset = dataset["train"]
shuffled = full_training_dataset.shuffle() #could be some biased order in the dataset so we randomize the order
training_dataset = shuffled.select(range(1000)) #select only the first 1000 for fine tuning

In [8]:
import bitsandbytes as bnb
from transformers import BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4", #normalized floating point 4
    bnb_4bit_compute_dtype ="float16" #data type for computation but not for storing model weights
)
#dequantization and quantization, compressing and decomppressing data into smaller memory formats and larger



1.   Load in model with automodel api, specify model name
2.   pass in quantization config
3. autotokenizer api

In [9]:
import transformers
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

MODEL_NAME = "NousResearch/Llama-2-7b-hf"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    device_map="auto")

model.config.use_cache = True

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side ="right"

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

# Format data for our model: input, output format needs to be concatinated into a big block of text to pass into the model for training
1.  store in var called combined
2.  tokenize it into numbers

In [10]:
def construct_datapoint(x):
  combined = x['input'] + x['output']
  return tokenizer(combined, padding =True)
training_dataset = training_dataset.map(construct_datapoint)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

input & output are the original columns
input_ids are the numbers (tokens) the tokenizer created

In [11]:
training_dataset

Dataset({
    features: ['input', 'output', 'input_ids', 'attention_mask'],
    num_rows: 1000
})

*  r is setting the width of matrices A&B --> controls complexity
*  alpha is like how much LoRA should we apply to each layer, almost like a multiplyer. High Alpha = High Influence. You are telling the model: "Trust this new adapter more than your old training."
*  target_modules are which layers we want to apply LoRA
*  Dropout prevents overfitting, dumb down model a little bit
*  task type is next token prediction (not translatopm pr image classification for example)

In [12]:
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=['q_proj','k_proj','down_proj','v_proj','gate_proj','o_proj','up_proj'],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config) #all layers except attention layers are frozen. this is the model we will train

generation_configuration = model.generation_config
generation_configuration.pad_token_id = tokenizer.eos_token_id
generation_configuration.eos_token_id = tokenizer.eos_token_id
generation_configuration.max_new_tokens = 256
generation_configuration.temperature = 0.7
generation_configuration.top_p = 0.9
generation_configuration.do_sample = True

#given a list of probs, we need to choose the next token


# Cant talk to this model like its a chatbot because it has not been fine tuned on any data yet, only been pretrained on the internet

This function is used to test the output of the model

In [13]:
def generate(prompt):

  generation_configuration.max_new_tokens = 20 #stop answering after 20 words

  encoded = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt").to(device) #model speaks math not english, need to convert my text string into a list of numbers, move numbers into gpu for speed
  with torch.inference_mode():
    out = model.generate( #this is the magic line
        input_ids=encoded,
        generation_config=generation_configuration,
        repetition_penalty=2.0)
    string_decoded = tokenizer.decode(out[0], clean_up_tokenization_spaces=True) #takes models output and turns those back into english for me
    print(string_decoded)

The Consequence: Now, the model sees that specific token and has a dual-identity crisis. It can't tell if the sentence is naturally over or if it's just looking at filler.

Why it's okay: You are feeding it a single prompt ("today I want to"). There is no "padding" needed because there are no other longer sentences in the batch to match length with.

When it matters: If you were generating text for 50 different prompts at the same time (batch inference), this confusion could cause the model to generate garbage or cut off sentences too early.


Dev did this because Llama 2 does not have a padding token so need to treat EOS token as the padding token

In [14]:
generate('today I want to')

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<s> today I want to talk about a subject that has been on my mind for quite some time: the fact of being un


In [15]:
generate('today I want to' + 'what is 5+5')

<s> today I want towhat is 5+5=10?
 everybody knows the answer of this question. And what about a more complicated mathematical


# Now we begin training

In [16]:
train_arguments = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    output_dir="fine_tuning"
)

trainer = transformers.Trainer(
    model=model, #model
    train_dataset=training_dataset, #training dataset
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False), #an object that will take care of any addtional data preprocessing
    args=train_arguments #training arguments
)

model.config.use_cache = False #now we are training and not generation so set to false

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


# took 27 minutes to run

In [17]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


TrainOutput(global_step=250, training_loss=0.6327050170898437, metrics={'train_runtime': 832.1894, 'train_samples_per_second': 1.202, 'train_steps_per_second': 0.3, 'total_flos': 5045715156418560.0, 'train_loss': 0.6327050170898437, 'epoch': 1.0})

In [18]:
evaluation_dataset = dataset['eval'].shuffle()

sample_sql_question = evaluation_dataset[1]['input']
correct_answer = evaluation_dataset[1]['output']

generate(sample_sql_question)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
Caching is incompatible with gradient checkpointing in LlamaDecoderLayer. Setting `past_key_values=None`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s> Below is an instruction that describes a SQL generation task, paired with an input that provides further context about the available table schemas. Write SQL code that appropriately answers the request.

### Instruction:
What is the Opposition in the First Round of the UEFA Cup with a Score of 3–1 (h), 2–0 (a)?

### Input:
CREATE TABLE table_name_66 (opposition VARCHAR, score VARCHAR, round VARCHAR, competition VARCHAR)

### Response:  SELECT belowbelow aboveBel Below Below Below Below Below Below Below Below Below Below Below Below Below Below Below


In [19]:
correct_answer

'SELECT opposition FROM table_name_66 WHERE round = "first round" AND competition = "uefa cup" AND score = "3–1 (h), 2–0 (a)"'